In [10]:
import numpy as np
import os
import matplotlib.pyplot as plt

In [5]:
data_dir = os.path.join("../data/")
data_fnames = os.listdir(data_dir)

In [7]:
for fname in data_fnames:
    full_path = os.path.join(data_dir, fname)
    A = np.load(full_path)

## Let's Look at Features/Structure of Data

In [13]:
print("Shape of A: %s" % (str(A.shape)))

Shape of A: (134801, 784)
